In [1]:
%load_ext autoreload
%autoreload 2
import sys
import os
current_dir = os.getcwd()
libs_path = os.path.abspath(os.path.join(current_dir, "..", "libs"))
if libs_path not in sys.path:
    sys.path.append(libs_path)

In [2]:
import pandas as pd
from IPython.display import display
from ipywidgets import interact
import numpy as np
pd.set_option('display.precision', 3)
import data_processing as dp

In [3]:
res=dp.load_datasets()

SelectMultiple(description='Датасеты:', layout=Layout(width='500px'), options={'.ipynb_checkpoints': '/workspa…

Button(description='ОК', style=ButtonStyle())

Output()

In [4]:
datasets = []

for fn in res.options:

    if res.options[fn] not in res.value:
        continue
    dataset = dp.Dataset(pd.read_csv(res.options[fn]), name=fn)

    dataset.target_features = ["Cover_Type"]
    datasets.append(dataset)

In [5]:
for d in datasets:
    print(d.name)



df_enhanced.csv_selected_features.csv_scaled.csv
df_enhanced_one-hot.csv_selected_features.csv_scaled.csv
V4_classification_lr3.csv_scaled.csv


In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Flatten, Input, MaxPooling1D, BatchNormalization
from tensorflow.keras.optimizers import SGD, Adam, RMSprop

def create_mlp_model(input_shape, num_classes, optimizer='adam', learning_rate=0.001, dense_units=[128, 64]):
    model = Sequential()
    model.add(Input(shape=(input_shape,)))
    for du in dense_units:
        model.add(Dense(du, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    if optimizer == 'sgd':
        opt = SGD(learning_rate=learning_rate)
    elif optimizer == 'adam':
        opt = Adam(learning_rate=learning_rate)
    elif optimizer == 'rmsprop':
        opt = RMSprop(learning_rate=learning_rate)
    else:
        opt = Adam(learning_rate=learning_rate)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def create_1d_cnn_model(input_shape, num_classes, filters=[128, 256],
                         kernel_size=3, pool_size=2, use_batch_norm=True, dense_units=[128, 64],
                         optimizer='adam', learning_rate=0.001):
    model = Sequential()
    model.add(Input(shape=(input_shape, 1))) # input_shape уже должен быть (num_features, 1)
    i=0
    for f in filters:
        model.add(Conv1D(filters=f, kernel_size=kernel_size, activation='relu', padding='same'))
        if use_batch_norm:
            model.add(BatchNormalization())
        # Добавляем MaxPooling1D, если есть смысл и достаточное количество признаков
        if input_shape > pool_size * (i + 1) : # Убедимся, что размерность не станет слишком маленькой
            model.add(MaxPooling1D(pool_size=pool_size))
        else:
            # Если MaxPooling1D может сделать размерность 0, пропускаем его
            # или можно настроить логику по-другому, например, убрать padding='same'
            # и добавить его только на первом слое
            pass # Пропускаем MaxPooling, если он приведет к 0 размерности
        i+=1
    model.add(Flatten())
    for du in dense_units:
        model.add(Dense(du, activation='relu'))
        if use_batch_norm: # Можно добавить BatchNormalization и здесь, но это опционально
            model.add(BatchNormalization())

    model.add(Dense(num_classes, activation='softmax'))

    # Выбор оптимизатора
    if optimizer == 'sgd':
        opt = SGD(learning_rate=learning_rate)
    elif optimizer == 'adam':
        opt = Adam(learning_rate=learning_rate)
    elif optimizer == 'rmsprop':
        opt = RMSprop(learning_rate=learning_rate)
    else:
        opt = Adam(learning_rate=learning_rate) # Default

    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model


    

2025-06-11 14:11:03.968085: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749651064.006520   93488 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749651064.022174   93488 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1749651064.105987   93488 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1749651064.106055   93488 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1749651064.106058   93488 computation_placer.cc:177] computation placer alr

In [7]:
from itertools import product

def all_combinations(args_dict):
    # Получаем имена аргументов и их возможные значения
    arg_names = args_dict.keys()
    values = args_dict.values()
    res = []
    # Перебираем все комбинации значений
    for combination in product(*values):
        # Создаем словарь с аргументами
        kwargs = dict(zip(arg_names, combination))
        # Вызываем функцию с текущей комбинацией
        res.append(kwargs)
    return res



In [ ]:

from scikeras.wrappers import KerasClassifier
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score, classification_report, confusion_matrix
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='loss', patience=5, restore_best_weights=True, verbose=0)

for d in datasets:
    d.target_features
    temp_name = d.name
    num_classes = int(d[d.target_features].nunique()) # ИСПРАВЛЕНИЕ: Количество уникальных классов
    assert num_classes == 7
    temp = []
    for i in range(num_classes):
        temp.append(d.target_features[0] + f"_{i}")
    #d.target_features = 
    d[temp] = to_categorical(d[d.target_features].values - 1, num_classes=7) # Эта строка остается
    d = d.drop(d.target_features, axis=1)
    d = dp.Dataset(d)
    d.target_features = temp
    d.name = temp_name
    
    d.split_data(random_state=1337)

    all_models = []
    params = {"optimizer" : ['sgd', 'adam', 'rmsprop'], "learning_rate" : [0.00001, 0.0001], "dense_units":[[128, 64, 32], [128, 64], [128], [64]]}
    params_1d_cnn = {"kernel_size" : [3], "pool_size": [2], "use_batch_norm": [False, True], "filters":[[128, 256], [64, 128, 256]]}
    params_mlp = {}
    params_default = {"input_shape": [d.shape[1] - num_classes], "num_classes" : [num_classes]}
    
    
    mlp_comb = all_combinations({**params_default, **params, **params_mlp})
    cnn_comb = all_combinations({**params_default, **params, **params_1d_cnn})
    #print(len(mlp_comb))
    print(len(cnn_comb))

    iii = 0
    #for el in mlp_comb:
    for el in cnn_comb:

        print(iii, el)
        iii+=1
        #model = create_mlp_model(**el)
        model = create_1d_cnn_model(**el)

        history = model.fit(d.X_train.values, d.Y_train.values, epochs=20, batch_size=64, callbacks=[early_stopping])
        with open('results2.txt', 'a', encoding='utf-8') as f:
            f.write(f"\n\n{d.name}; {el};")

        

        Y_train_real = model.predict(d.X_train.values, verbose=0)
        Y_test_real = model.predict(d.X_test.values, verbose=0)
        Y_valid_real = model.predict(d.X_valid.values, verbose=0)
       # print(np.isnan(Y_train_real).any())
       # print(np.isnan(Y_test_real).any())
       # print(np.isnan(Y_valid_real).any())

        AAA = [[np.argmax(Y_train_real, axis=1), np.argmax(d.Y_train.values, axis=1), "train"], [np.argmax(Y_test_real, axis=1), np.argmax(d.Y_test.values, axis=1), "test"], [np.argmax(Y_valid_real, axis=1), np.argmax(d.Y_valid.values, axis=1), "valid"]]
        with open('results2.txt', 'a', encoding='utf-8') as f:
            for viborka in AAA:
                try:
                    #print(type(viborka[0]), type(viborka[1]))
                    #print(accuracy_score(viborka[0], viborka[1]))
                    f.write(f"{viborka[2]}; acc= {accuracy_score(viborka[0], viborka[1])};\n")
                except Exception as e:
                    print(e)
                    f.write(f"{viborka[2]}; acc= accuracy_score_error;\n")
                try:

                    f.write(f"{viborka[2]}; bal_acc= {balanced_accuracy_score(viborka[0], viborka[1])};\n")
                except:
                    f.write(f"{viborka[2]}; bal_acc= balanced_accuracy_score_error;\n")

                try:

                    f.write(f"{viborka[2]}; f1_macro= {f1_score(viborka[0], viborka[1], average='macro', zero_division=0)};\n")
                except:
                    f.write(f"{viborka[2]}; f1_macro= f1_score_error;\n")

                try:

                    f.write(f"{viborka[2]}; f1_weighted= {f1_score(viborka[0], viborka[1], average='weighted', zero_division=0)};\n")
                except:
                    f.write(f"{viborka[2]}; f1_weighted= f1_score_error;\n")

                try:
                    f.write(f"{viborka[2]}; classification_report= {classification_report(viborka[0], viborka[1], target_names=[f'Class {i+1}' for i in range(num_classes)], zero_division=0)};\n")
                except:
                    f.write(f"{viborka[2]}; classification_report= classification_report_error;\n")

    


/tmp/ipykernel_93488/4209599979.py:11: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  num_classes = int(d[d.target_features].nunique()) # ИСПРАВЛЕНИЕ: Количество уникальных классов


96
0 {'input_shape': 12, 'num_classes': 7, 'optimizer': 'sgd', 'learning_rate': 1e-05, 'dense_units': [128, 64, 32], 'kernel_size': 3, 'pool_size': 2, 'use_batch_norm': False, 'filters': [128, 256]}
Epoch 1/20
5528/5528 ━━━━━━━━━━━━━━━━━━━━ 16s 3ms/step - accuracy: 0.1820 - loss: 1.9533
Epoch 2/20
5528/5528 ━━━━━━━━━━━━━━━━━━━━ 14s 3ms/step - accuracy: 0.4989 - loss: 1.9162
Epoch 3/20
5528/5528 ━━━━━━━━━━━━━━━━━━━━ 15s 3ms/step - accuracy: 0.4988 - loss: 1.8907
Epoch 4/20
5528/5528 ━━━━━━━━━━━━━━━━━━━━ 16s 3ms/step - accuracy: 0.4988 - loss: 1.8671
Epoch 5/20
5528/5528 ━━━━━━━━━━━━━━━━━━━━ 16s 3ms/step - accuracy: 0.4990 - loss: 1.8446
Epoch 6/20
5528/5528 ━━━━━━━━━━━━━━━━━━━━ 16s 3ms/step - accuracy: 0.5009 - loss: 1.8225
Epoch 7/20
5528/5528 ━━━━━━━━━━━━━━━━━━━━ 16s 3ms/step - accuracy: 0.5029 - loss: 1.8011
Epoch 8/20
5528/5528 ━━━━━━━━━━━━━━━━━━━━ 16s 3ms/step - accuracy: 0.5033 - loss: 1.7800
Epoch 9/20
5528/5528 ━━━━━━━━━━━━━━━━━━━━ 16s 3ms/step - accuracy: 0.5020 - loss: 1.7589


2025-06-11 14:22:32.546752: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_62', 12 bytes spill stores, 12 bytes spill loads

2025-06-11 14:22:32.643999: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_62', 4 bytes spill stores, 4 bytes spill loads

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:2776: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:2776: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:2776: UserWarning: y_pred contains classes not in y_true
  warnings.warn(

1 {'input_shape': 12, 'num_classes': 7, 'optimizer': 'sgd', 'learning_rate': 1e-05, 'dense_units': [128, 64, 32], 'kernel_size': 3, 'pool_size': 2, 'use_batch_norm': False, 'filters': [64, 128, 256]}
Epoch 1/20


2025-06-11 14:23:07.484915: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_524', 416 bytes spill stores, 416 bytes spill loads

2025-06-11 14:23:07.498281: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_524', 604 bytes spill stores, 512 bytes spill loads

2025-06-11 14:23:07.570330: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_524', 680 bytes spill stores, 548 bytes spill loads



...

66 {'input_shape': 12, 'num_classes': 7, 'optimizer': 'rmsprop', 'learning_rate': 1e-05, 'dense_units': [128, 64, 32], 'kernel_size': 3, 'pool_size': 2, 'use_batch_norm': True, 'filters': [128, 256]}
Epoch 1/20
5528/5528 ━━━━━━━━━━━━━━━━━━━━ 21s 3ms/step - accuracy: 0.6189 - loss: 1.3272
Epoch 2/20
5528/5528 ━━━━━━━━━━━━━━━━━━━━ 17s 3ms/step - accuracy: 0.7323 - loss: 0.8913
Epoch 3/20
5528/5528 ━━━━━━━━━━━━━━━━━━━━ 17s 3ms/step - accuracy: 0.7453 - loss: 0.7456
Epoch 4/20
5528/5528 ━━━━━━━━━━━━━━━━━━━━ 17s 3ms/step - accuracy: 0.7540 - loss: 0.6706
Epoch 5/20
5528/5528 ━━━━━━━━━━━━━━━━━━━━ 17s 3ms/step - accuracy: 0.7594 - loss: 0.6263
Epoch 6/20
5528/5528 ━━━━━━━━━━━━━━━━━━━━ 17s 3ms/step - accuracy: 0.7631 - loss: 0.5982
Epoch 7/20
5528/5528 ━━━━━━━━━━━━━━━━━━━━ 16s 3ms/step - accuracy: 0.7670 - loss: 0.5754
Epoch 8/20
5528/5528 ━━━━━━━━━━━━━━━━━━━━ 17s 3ms/step - accuracy: 0.7721 - loss: 0.5558
Epoch 9/20
5528/5528 ━━━━━━━━━━━━━━━━━━━━ 17s 3ms/step - accuracy: 0.7719 - loss: 0.5493

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:2776: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:2776: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:2776: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


69 {'input_shape': 12, 'num_classes': 7, 'optimizer': 'rmsprop', 'learning_rate': 1e-05, 'dense_units': [128, 64], 'kernel_size': 3, 'pool_size': 2, 'use_batch_norm': False, 'filters': [64, 128, 256]}
Epoch 1/20
5528/5528 ━━━━━━━━━━━━━━━━━━━━ 20s 3ms/step - accuracy: 0.4928 - loss: 1.3737
Epoch 2/20
5528/5528 ━━━━━━━━━━━━━━━━━━━━ 17s 3ms/step - accuracy: 0.6196 - loss: 0.8779
Epoch 3/20
5528/5528 ━━━━━━━━━━━━━━━━━━━━ 17s 3ms/step - accuracy: 0.6854 - loss: 0.7866
Epoch 4/20
5528/5528 ━━━━━━━━━━━━━━━━━━━━ 18s 3ms/step - accuracy: 0.7024 - loss: 0.7509
Epoch 5/20
5528/5528 ━━━━━━━━━━━━━━━━━━━━ 17s 3ms/step - accuracy: 0.7062 - loss: 0.7326
Epoch 6/20
5528/5528 ━━━━━━━━━━━━━━━━━━━━ 17s 3ms/step - accuracy: 0.7091 - loss: 0.7126
Epoch 7/20
5528/5528 ━━━━━━━━━━━━━━━━━━━━ 17s 3ms/step - accuracy: 0.7103 - loss: 0.6962
Epoch 8/20
5528/5528 ━━━━━━━━━━━━━━━━━━━━ 17s 3ms/step - accuracy: 0.7136 - loss: 0.6831
Epoch 9/20
5528/5528 ━━━━━━━━━━━━━━━━━━━━ 17s 3ms/step - accuracy: 0.7125 - loss: 0.674

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:2776: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:2776: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:2776: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


70 {'input_shape': 12, 'num_classes': 7, 'optimizer': 'rmsprop', 'learning_rate': 1e-05, 'dense_units': [128, 64], 'kernel_size': 3, 'pool_size': 2, 'use_batch_norm': True, 'filters': [128, 256]}
Epoch 1/20
5528/5528 ━━━━━━━━━━━━━━━━━━━━ 21s 3ms/step - accuracy: 0.6412 - loss: 1.2021
Epoch 2/20
5528/5528 ━━━━━━━━━━━━━━━━━━━━ 17s 3ms/step - accuracy: 0.7365 - loss: 0.7822
Epoch 3/20
5528/5528 ━━━━━━━━━━━━━━━━━━━━ 17s 3ms/step - accuracy: 0.7495 - loss: 0.6856
Epoch 4/20
5528/5528 ━━━━━━━━━━━━━━━━━━━━ 17s 3ms/step - accuracy: 0.7560 - loss: 0.6303
Epoch 5/20
5528/5528 ━━━━━━━━━━━━━━━━━━━━ 17s 3ms/step - accuracy: 0.7619 - loss: 0.5990
Epoch 6/20
5528/5528 ━━━━━━━━━━━━━━━━━━━━ 17s 3ms/step - accuracy: 0.7659 - loss: 0.5737
Epoch 7/20
 763/5528 ━━━━━━━━━━━━━━━━━━━━ 14s 3ms/step - accuracy: 0.7660 - loss: 0.5608

In [14]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))

[]
